In [11]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import base64
import pandas as pd
from AnimalShelter import AnimalShelter

# Database connection
username = "aacuser"
password = "SNHU1234"
db = AnimalShelter(username, password)

# Get full dataset
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Logo setup
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# App and layout
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Center(html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                 style={'height': '100px'}), href='https://www.snhu.edu', target='_blank')),
    html.Center(html.B(html.H2("Created by Koda Ermini"))),
    html.Center(html.H1('Grazioso Salvare Rescue Dog Dashboard')),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'marginRight': '15px'}
        )
    ], style={'textAlign': 'center'}),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action='native',
        sort_action='native',
        style_table={'overflowX': 'auto'}
    ),
    html.Br(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])


In [12]:

@app.callback(Output('datatable-id', 'data'),
              Input('filter-type', 'value'))
def update_dashboard(filter_type):
    query = {}
    if filter_type == 'water':
        query = {
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'mountain':
        query = {
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif filter_type == 'disaster':
        query = {
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    results = db.read(query)
    dff = pd.DataFrame.from_records(results)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)
    return dff.to_dict('records')


In [13]:

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [html.P("No data to display.")]
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff, names='breed', title='Breed Representation in Results')
    return [dcc.Graph(figure=fig)]


In [14]:
app.run_server(mode='external', debug=True)

Dash app running on http://127.0.0.1:13358/
